<a href="https://colab.research.google.com/github/positivejmk/2021-2-practice/blob/master/Kmeans_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#1. Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv("/content/drive/MyDrive/BA team proj/공모전/nh_merge_final.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.shape

(681472, 49)

In [6]:
df.head()

,Unnamed: 0,Unnamed: 0.1,act_id,iem_cd,byn_dt,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p,weekday,weekday_vc,holdday_category_mean,holdday_category_sum,min,max,duration,TotalSales,OrderCount,AvgOrderValue,sales_intervaldays,price_perstock_mean,금가격,원달러환율,KOSPI지수 (1980.01.04=100),"거래량(주식시장, 잠정치) (만주)","거래대금(주식시장 , 잠정치) (억원)","외국인 순매수(주식시장, 잠정치) (억원)","주식시장-거래량(만주, 시간외거래분 포함) (만주)","주식시장-거래대금(억원, 시간외거래분 포함) (억원)","시가총액(주식시장, 잠정치) (억원)",KOSDAQ지수 (1996.07.01=1000),"거래량(만주 : 코스닥시장, 잠정치) (만주)","거래대금(억원 : 코스닥시장, 잠정치) (억원)","외국인 순매수(코스닥시장, 잠정치) (억원)"
0,0,0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,1,9,3,2,2,9,5,5,8,101,1,1,1,20180726,274.0,11782000.0,5000.0,43000.0,3,140505,23.279391,267713,2016-01-04,2020-11-12,1774,36209.0,76,476.434211,23.342105,2.325581e-05,1228.59,1120.0,2289.06,29532,61114,1285,30333,63571,15309140,765.15,53644,35128,591
1,1,6,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,80,1,9,3,2,2,9,5,5,8,1361,9,1,1,20180131,2.0,4990000.0,5000.0,2495000.0,2,134947,25.139455,1301721,2016-01-04,2020-11-12,1774,36209.0,76,476.434211,23.342105,4.008016e-07,1342.10,1067.0,2566.46,39690,107348,-6630,40305,108426,16685790,913.57,119526,75432,-1782
2,2,15,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,5,1,9,3,2,2,9,5,5,8,2530,12,2,99,20180517,786.0,14619600.0,1000.0,18600.0,3,140505,21.796607,760658,2016-01-04,2020-11-12,1774,36209.0,76,476.434211,23.342105,5.376344e-05,1287.43,1081.5,2448.45,58898,80783,-2708,59677,81905,16345480,855.62,103744,54606,729
3,3,18,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,2020-11-12,22,1,9,3,2,2,9,5,5,8,1969,8,2,1,20201112,60.0,462000.0,500.0,7700.0,3,140505,17.620062,618147,2016-01-04,2020-11-12,1774,36209.0,76,476.434211,23.342105,1.298701e-04,1867.50,1115.5,2475.62,125045,157200,4898,127869,164200,16983290,840.08,206889,110048,63
4,4,20,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,2018-09-05,324,1,9,3,2,2,9,5,5,8,1696,10,3,99,20180905,183.0,8125200.0,5000.0,44400.0,2,134947,16.920504,2991782,2016-01-04,2020-11-12,1774,36209.0,76,476.434211,23.342105,2.252252e-05,1195.25,1123.5,2291.77,35203,52202,-980,35819,53905,15333130,820.00,66369,43497,256


#Per act_id Data

In [17]:
id_df = df.groupby('act_id').agg({
    'sales_intervaldays': min,
    'bnc_qty' : sum,
    'byn_dt' : lambda x: x.nunique()
})

id_df.columns = ['Intervaldays', 'TotalSales', 'OrderCount']

id_df

,Intervaldays,TotalSales,OrderCount
act_id,,,
0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c5467ece45d5b35e99912a,34.875000,517.0,16
0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa9b6e6efa399d0bdf31,8.964286,11925.0,28
0008d1bfac569bdb998897e2eb8519dec06b9277a686f4c6e06de582c9791b35,13.000000,33085.0,51
000a83a961cbcbce40673aed998effbd462fb10dcac8e64c6fa720f2150d3f5a,160.666667,3.0,3
000b92ca83a96312a0909b1f489340f4d399c86555012ffe2c41952ae43dc741,36.666667,4.0,3
...,...,...,...
ffdccfb9dc24cf41843f6e7f34704ccb73ac81acc29aa39b043fe7f90959e5f3,2.689286,812033.0,280
ffecc8e2823943ef1a7b0d9c6512f716a88032a1bdf7bace472b4365f5028535,9.795918,63726.0,49
ffee399294db1c721e010cbf3dae5c8a1f140dbaf2dcae96e7cd290e431aee0a,30.315789,4483.0,19


In [19]:
id_df.describe()

,Intervaldays,TotalSales,OrderCount
count,9337.000000,9.337000e+03,9337.000000
mean,25.897282,3.635851e+04,44.962515
std,39.633256,1.417177e+05,64.143509
min,0.000000,1.000000e+00,1.000000
25%,5.711111,5.030000e+02,8.000000
50%,12.800000,3.500000e+03,22.000000
75%,29.315789,1.981700e+04,55.000000
max,551.000000,4.093110e+06,738.000000


In [20]:
# rank(): 각 열마다 순위 값을 반환, index 축을 기준으로 보여줌
# 아무것도 지정하지 않으면 기본값인 오름 차순으로 순위를 정함 (낮은 숫자에서 높은 순으로)
# method: 동일값의 경우 순위 지정하는 방법 
#         average: average rank of the group (default)
#         min: lowest rank in the group
#         max: highest rank in the group
#         first: ranks assigned in order they appear in the array
#         dense: like ‘min’, but rank always increases by 1 between groups.

rank_df = id_df.rank(method='first')

In [21]:
rank_df.head()

,Intervaldays,TotalSales,OrderCount
act_id,,,
0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c5467ece45d5b35e99912a,7419.0,2363.0,3776.0
0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa9b6e6efa399d0bdf31,3587.0,6361.0,5260.0
0008d1bfac569bdb998897e2eb8519dec06b9277a686f4c6e06de582c9791b35,4708.0,7659.0,6797.0
000a83a961cbcbce40673aed998effbd462fb10dcac8e64c6fa720f2150d3f5a,9171.0,76.0,892.0
000b92ca83a96312a0909b1f489340f4d399c86555012ffe2c41952ae43dc741,7524.0,100.0,893.0


In [22]:
rank_df.describe()

,Intervaldays,TotalSales,OrderCount
count,9337.000000,9337.000000,9337.000000
mean,4669.000000,4669.000000,4669.000000
std,2695.504065,2695.504065,2695.504065
min,1.000000,1.000000,1.000000
25%,2335.000000,2335.000000,2335.000000
50%,4669.000000,4669.000000,4669.000000
75%,7003.000000,7003.000000,7003.000000
max,9337.000000,9337.000000,9337.000000


In [26]:
# 군집분석 전 데이터 스케일 표준화 
scaled_df = (rank_df - rank_df.mean()) / rank_df.std()

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(rank_df)

In [27]:
scaled_df

,Intervaldays,TotalSales,OrderCount
act_id,,,
0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c5467ece45d5b35e99912a,1.020217,-0.855499,-0.331292
0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa9b6e6efa399d0bdf31,-0.401409,0.627712,0.219254
0008d1bfac569bdb998897e2eb8519dec06b9277a686f4c6e06de582c9791b35,0.014469,1.109254,0.789463
000a83a961cbcbce40673aed998effbd462fb10dcac8e64c6fa720f2150d3f5a,1.670189,-1.703948,-1.401222
000b92ca83a96312a0909b1f489340f4d399c86555012ffe2c41952ae43dc741,1.059171,-1.695045,-1.400851
...,...,...,...
ffdccfb9dc24cf41843f6e7f34704ccb73ac81acc29aa39b043fe7f90959e5f3,-1.406416,1.712110,1.682060
ffecc8e2823943ef1a7b0d9c6512f716a88032a1bdf7bace472b4365f5028535,-0.304952,1.331105,0.774252
ffee399294db1c721e010cbf3dae5c8a1f140dbaf2dcae96e7cd290e431aee0a,0.898533,0.121684,-0.113522


In [28]:
scaled_df.describe()

,Intervaldays,TotalSales,OrderCount
count,9.337000e+03,9.337000e+03,9.337000e+03
mean,-5.166455e-18,-2.047557e-17,-1.004902e-17
std,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.731773e+00,-1.731773e+00,-1.731773e+00
25%,-8.658863e-01,-8.658863e-01,-8.658863e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.658863e-01,8.658863e-01,8.658863e-01
max,1.731773e+00,1.731773e+00,1.731773e+00


#2. Customer Segmentation via K-Means Clustering